In [91]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [108]:
app_data = pd.read_csv("Google-Playstore.csv")
app_data.head()

,App Name,App Id,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,Price,...,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice,Scraped Time
0,Gakondo,com.ishakwe.gakondo,Adventure,0.0,0.0,10+,10.0,15,True,0.0,...,https://beniyizibyose.tk/#/,jean21101999@gmail.com,"Feb 26, 2020","Feb 26, 2020",Everyone,https://beniyizibyose.tk/projects/,False,False,False,2021-06-15 20:19:35
1,Ampere Battery Info,com.webserveis.batteryinfo,Tools,4.4,64.0,"5,000+",5000.0,7662,True,0.0,...,https://webserveis.netlify.app/,webserveis@gmail.com,"May 21, 2020","May 06, 2021",Everyone,https://dev4phones.wordpress.com/licencia-de-uso/,True,False,False,2021-06-15 20:19:35
2,Vibook,com.doantiepvien.crm,Productivity,0.0,0.0,50+,50.0,58,True,0.0,...,NaN,vnacrewit@gmail.com,"Aug 9, 2019","Aug 19, 2019",Everyone,https://www.vietnamairlines.com/vn/en/terms-an...,False,False,False,2021-06-15 20:19:35
3,Smart City Trichy Public Service Vehicles 17UC...,cst.stJoseph.ug17ucs548,Communication,5.0,5.0,10+,10.0,19,True,0.0,...,http://www.climatesmarttech.com/,climatesmarttech2@gmail.com,"Sep 10, 2018","Oct 13, 2018",Everyone,NaN,True,False,False,2021-06-15 20:19:35
4,GROW.me,com.horodyski.grower,Tools,0.0,0.0,100+,100.0,478,True,0.0,...,http://www.horodyski.com.pl,rmilekhorodyski@gmail.com,"Feb 21, 2020","Nov 12, 2018",Everyone,http://www.horodyski.com.pl,False,False,False,2021-06-15 20:19:35


In [79]:
app_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2312944 entries, 0 to 2312943
Data columns (total 24 columns):
 #   Column             Dtype  
---  ------             -----  
 0   App Name           object 
 1   App Id             object 
 2   Category           object 
 3   Rating             float64
 4   Rating Count       float64
 5   Installs           object 
 6   Minimum Installs   float64
 7   Maximum Installs   int64  
 8   Free               bool   
 9   Price              float64
 10  Currency           object 
 11  Size               object 
 12  Minimum Android    object 
 13  Developer Id       object 
 14  Developer Website  object 
 15  Developer Email    object 
 16  Released           object 
 17  Last Updated       object 
 18  Content Rating     object 
 19  Privacy Policy     object 
 20  Ad Supported       bool   
 21  In App Purchases   bool   
 22  Editors Choice     bool   
 23  Scraped Time       object 
dtypes: bool(4), float64(4), int64(1), object(15)
memor

In [94]:
app_data.shape

(2312944, 24)

In [109]:
app_data.drop(['App Id', 'Currency', 'Size', 'Minimum Android', 'Developer Id', 'Developer Website', 'Developer Email', 'Privacy Policy', 'Editors Choice', 'Scraped Time'], axis=1, inplace=True)
app_data.head()

,App Name,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,Price,Released,Last Updated,Content Rating,Ad Supported,In App Purchases
0,Gakondo,Adventure,0.0,0.0,10+,10.0,15,True,0.0,"Feb 26, 2020","Feb 26, 2020",Everyone,False,False
1,Ampere Battery Info,Tools,4.4,64.0,"5,000+",5000.0,7662,True,0.0,"May 21, 2020","May 06, 2021",Everyone,True,False
2,Vibook,Productivity,0.0,0.0,50+,50.0,58,True,0.0,"Aug 9, 2019","Aug 19, 2019",Everyone,False,False
3,Smart City Trichy Public Service Vehicles 17UC...,Communication,5.0,5.0,10+,10.0,19,True,0.0,"Sep 10, 2018","Oct 13, 2018",Everyone,True,False
4,GROW.me,Tools,0.0,0.0,100+,100.0,478,True,0.0,"Feb 21, 2020","Nov 12, 2018",Everyone,False,False


In [96]:
print(app_data.isnull().sum())

App Name                2
Category                0
Rating              22883
Rating Count        22883
Installs              107
Minimum Installs      107
Maximum Installs        0
Free                    0
Price                   0
Released            71053
Last Updated            0
Content Rating          0
Ad Supported            0
In App Purchases        0
dtype: int64


In [63]:
app_data.shape

(2312944, 14)

In [110]:
app_data.dropna(subset=['Rating', 'Rating Count', 'Installs'], inplace=True)
app_data.shape

(2290061, 14)

In [82]:
app_data['Content Rating'].value_counts()

Everyone           2002651
Teen                194103
Mature 17+           59740
Everyone 10+         33280
Unrated                154
Adults only 18+        133
Name: Content Rating, dtype: int64

In [111]:
app_data['Content Rating'] = app_data['Content Rating'].replace('Unrated',"Everyone")

#Cleaning other values just to include Everyone, Teens and Adult 

app_data['Content Rating'] = app_data['Content Rating'].replace('Mature 17+',"Adults")
app_data['Content Rating'] = app_data['Content Rating'].replace('Adults only 18+',"Adults")
app_data['Content Rating'] = app_data['Content Rating'].replace('Everyone 10+',"Everyone")

app_data['Content Rating'].value_counts()

Everyone    2036085
Teen         194103
Adults        59873
Name: Content Rating, dtype: int64

In [112]:
app_data.Installs = app_data.Installs.str.replace(',','')
app_data.Installs = app_data.Installs.str.replace('+','')
app_data['Installs'] = pd.to_numeric(app_data['Installs'])

/var/folders/z6/589dnrx92014vt29m3134f4h0000gn/T/ipykernel_43968/2616268278.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  app_data.Installs = app_data.Installs.str.replace('+','')


In [69]:
app_data.head()

,App Name,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,Price,Released,Last Updated,Content Rating,Ad Supported,In App Purchases
0,Gakondo,Adventure,0.0,0.0,10,10.0,15,True,0.0,"Feb 26, 2020","Feb 26, 2020",Everyone,False,False
1,Ampere Battery Info,Tools,4.4,64.0,5000,5000.0,7662,True,0.0,"May 21, 2020","May 06, 2021",Everyone,True,False
2,Vibook,Productivity,0.0,0.0,50,50.0,58,True,0.0,"Aug 9, 2019","Aug 19, 2019",Everyone,False,False
3,Smart City Trichy Public Service Vehicles 17UC...,Communication,5.0,5.0,10,10.0,19,True,0.0,"Sep 10, 2018","Oct 13, 2018",Everyone,True,False
4,GROW.me,Tools,0.0,0.0,100,100.0,478,True,0.0,"Feb 21, 2020","Nov 12, 2018",Everyone,False,False


In [113]:
app_data['Category'] = pd.factorize(app_data['Category'])[0].astype(int)
app_data['Content Rating'] = pd.factorize(app_data['Content Rating'])[0].astype(int)
app_data['Ad Supported'] = pd.factorize(app_data['Ad Supported'])[0].astype(int)
app_data['In App Purchases'] = pd.factorize(app_data['In App Purchases'])[0].astype(int)
app_data['Free'] = pd.factorize(app_data['Free'])[0].astype(int)
app_data.head()

,App Name,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,Price,Released,Last Updated,Content Rating,Ad Supported,In App Purchases
0,Gakondo,0,0.0,0.0,10,10.0,15,0,0.0,"Feb 26, 2020","Feb 26, 2020",0,0,0
1,Ampere Battery Info,1,4.4,64.0,5000,5000.0,7662,0,0.0,"May 21, 2020","May 06, 2021",0,1,0
2,Vibook,2,0.0,0.0,50,50.0,58,0,0.0,"Aug 9, 2019","Aug 19, 2019",0,0,0
3,Smart City Trichy Public Service Vehicles 17UC...,3,5.0,5.0,10,10.0,19,0,0.0,"Sep 10, 2018","Oct 13, 2018",0,1,0
4,GROW.me,1,0.0,0.0,100,100.0,478,0,0.0,"Feb 21, 2020","Nov 12, 2018",0,0,0


In [114]:
X = app_data.drop(['App Name', 'Rating', 'Rating Count', 'Minimum Installs', 'Maximum Installs', 'Price', 'Released', 'Last Updated'], axis=1)
X.head()

,Category,Installs,Free,Content Rating,Ad Supported,In App Purchases
0,0,10,0,0,0,0
1,1,5000,0,0,1,0
2,2,50,0,0,0,0
3,3,10,0,0,1,0
4,1,100,0,0,0,0


In [115]:
y = app_data['Rating'].values.round()
print(y[0:4])

[0. 4. 0. 5.]


In [116]:
std_scaler = StandardScaler()
X['Installs'] = std_scaler.fit_transform(X[['Installs']])
X.head()

,Category,Installs,Free,Content Rating,Ad Supported,In App Purchases
0,0,-0.012172,0,0,0,0
1,1,-0.011844,0,0,1,0
2,2,-0.012170,0,0,0,0
3,3,-0.012172,0,0,1,0
4,1,-0.012167,0,0,0,0


In [117]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8)

In [118]:
rf_model = RandomForestClassifier()
rf_model.fit(X_train,y_train)
y_pred = rf_model.predict(X_test)
rf_acc = accuracy_score(y_pred,y_test)*100
print("Accuracy =",round(rf_acc,2),"%")

Accuracy = 69.26 %
